In [66]:

from selenium.webdriver import Chrome
from selenium.webdriver.chrome.options import Options
import requests
from bs4 import BeautifulSoup
import pandas as pd
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.keys import Keys
import time
import re

import shutil
from datetime import datetime
from dateutil.relativedelta import relativedelta
from datetime import timezone
import os
import ssl

from selenium.webdriver.common.action_chains import ActionChains
from IPython.display import clear_output

from selenium import webdriver
from selenium.webdriver.common.desired_capabilities import DesiredCapabilities
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
import random
from urllib.parse import quote



def open_browser():
    opts = Options()
    opts.headless = False
    #user:pass@10.10.10.10:8000
    #PROXY='114.7.27.98:8080'
    #opts.add_argument('--proxy-server=%s' % PROXY)
    
    #driver = webdriver.Chrome(desired_capabilities=capa)
    driver = webdriver.Chrome(ChromeDriverManager().install(), options=opts) 
           
    return driver





<h2>Loading All Companies Info</h2>

In [8]:
df_companies=pd.read_csv("company_info.csv",low_memory=False)


In [11]:
print("Shape:",df_companies.shape)
print("Columns:",df_companies.columns)

Shape: (29499, 37)
Columns: Index(['Unnamed: 0', 'symbol', 'price', 'beta', 'volAvg', 'mktCap', 'lastDiv',
       'range', 'changes', 'companyName', 'currency', 'cik', 'isin', 'cusip',
       'exchange', 'exchangeShortName', 'industry', 'website', 'description',
       'ceo', 'sector', 'country', 'fullTimeEmployees', 'phone', 'address',
       'city', 'state', 'zip', 'dcfDiff', 'dcf', 'image', 'ipoDate',
       'defaultImage', 'isEtf', 'isActivelyTrading', 'isAdr', 'isFund'],
      dtype='object')


1. Filtering companies whose currency of trading is USD
2. Selecting top 2000 companies by marketcap

In [34]:
df_companies_filtered=df_companies[df_companies['currency']=='USD'].sort_values(by=['mktCap'],ascending=[False]).head(2000).reset_index(drop=True)

In [35]:
df_companies_filtered.head()

,Unnamed: 0,symbol,price,beta,volAvg,mktCap,lastDiv,range,changes,companyName,...,zip,dcfDiff,dcf,image,ipoDate,defaultImage,isEtf,isActivelyTrading,isAdr,isFund
0,11,MSFT,335.95,0.802645,23534312,2.522306e+12,2.240,208.16-338.79,-1.039978,Microsoft Corporation,...,98052-6399,-60.78,338.534,https://financialmodelingprep.com/image-stock/...,1986-03-13,False,False,True,False,False
1,10,AAPL,150.81,1.222222,76073748,2.474249e+12,0.865,112.59-157.26,0.369995,Apple Inc.,...,95014,89.92,152.609,https://financialmodelingprep.com/image-stock/...,1980-12-12,False,False,True,False,False
2,24437,MSF.BR,325.00,0.862337,124,2.440093e+12,2.240,208.55-354.95,-12.200012,Microsoft Corporation,...,98052-6399,NaN,0.000,https://financialmodelingprep.com/image-stock/...,NaN,False,False,True,False,False
3,513,GOOG,2984.97,1.025925,1105707,1.979098e+12,0.000,1699.0-3020.69,-2.060059,Alphabet Inc.,...,94043,376.88,3025.900,https://financialmodelingprep.com/image-stock/...,2004-08-19,False,False,True,False,False
4,394,GOOGL,2978.37,1.025925,1455134,1.978969e+12,0.000,1694.0-3012.3,-2.250000,Alphabet Inc.,...,94043,344.37,3018.510,https://financialmodelingprep.com/image-stock/...,2004-08-19,False,False,True,False,False


In [36]:
df_companies_filtered.describe()

,Unnamed: 0,price,beta,volAvg,mktCap,lastDiv,changes,cik,dcfDiff,dcf
count,2000.000000,2000.000000,2000.000000,2.000000e+03,2.000000e+03,2000.000000,2000.000000,1.195000e+03,5.910000e+02,1363.000000
mean,7426.580000,361.706143,0.849110,2.240586e+06,5.337600e+10,1.395824,-1.745393,1.031255e+06,-6.799250e+08,796.739395
std,7539.905001,9652.825017,0.710660,6.214752e+06,1.393428e+11,6.219682,79.127607,5.497775e+05,1.652933e+10,14114.585701
min,0.000000,0.650000,-7.533005,0.000000e+00,8.201438e+09,0.000000,-3520.000000,1.800000e+03,-4.018360e+11,-193.559000
25%,843.750000,38.462500,0.208188,0.000000e+00,1.330322e+10,0.000000,-0.462505,8.114825e+05,-5.548000e+01,0.000000
50%,3962.500000,78.250000,0.888915,4.170390e+05,2.358917e+10,0.561000,-0.009999,1.067983e+06,-1.118000e+01,64.982900
75%,14021.250000,164.312500,1.284155,1.861823e+06,5.025814e+10,1.601750,0.389999,1.477584e+06,6.882500e+01,169.113500
max,29460.000000,431680.000000,5.026002,9.835381e+07,2.522306e+12,259.000000,112.339840,1.868726e+06,5.796180e+06,465298.000000


In [39]:
df_companies_filtered.to_csv("FilteredCompanies.csv")
df_companies_filtered.head()

,Unnamed: 0,symbol,price,beta,volAvg,mktCap,lastDiv,range,changes,companyName,...,zip,dcfDiff,dcf,image,ipoDate,defaultImage,isEtf,isActivelyTrading,isAdr,isFund
0,11,MSFT,335.95,0.802645,23534312,2.522306e+12,2.240,208.16-338.79,-1.039978,Microsoft Corporation,...,98052-6399,-60.78,338.534,https://financialmodelingprep.com/image-stock/...,1986-03-13,False,False,True,False,False
1,10,AAPL,150.81,1.222222,76073748,2.474249e+12,0.865,112.59-157.26,0.369995,Apple Inc.,...,95014,89.92,152.609,https://financialmodelingprep.com/image-stock/...,1980-12-12,False,False,True,False,False
2,24437,MSF.BR,325.00,0.862337,124,2.440093e+12,2.240,208.55-354.95,-12.200012,Microsoft Corporation,...,98052-6399,NaN,0.000,https://financialmodelingprep.com/image-stock/...,NaN,False,False,True,False,False
3,513,GOOG,2984.97,1.025925,1105707,1.979098e+12,0.000,1699.0-3020.69,-2.060059,Alphabet Inc.,...,94043,376.88,3025.900,https://financialmodelingprep.com/image-stock/...,2004-08-19,False,False,True,False,False
4,394,GOOGL,2978.37,1.025925,1455134,1.978969e+12,0.000,1694.0-3012.3,-2.250000,Alphabet Inc.,...,94043,344.37,3018.510,https://financialmodelingprep.com/image-stock/...,2004-08-19,False,False,True,False,False


### Open Browser

In [107]:
try:
    driver.quit()
except:
    pass

driver=open_browser()

[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 95.0.4638
[WDM] - Get LATEST driver version for 95.0.4638


[WDM] - Driver [C:\Users\shriv\.wdm\drivers\chromedriver\win32\95.0.4638.69\chromedriver.exe] found in cache


<h2>Functions

In [108]:


#print(job)

def fetch_job_info(job):
    job_title=None
    job_link=None
    job_company_name=None
    company_link=None
    company_rating=None
    job_location=None
    shortDescription=None
    posting_date=None
    salary_text=None


    try:
        job_title=str(job.find("h2",{"class":"jobTitle"}).find_all("span")[-1]['title']).strip()
    except Exception as e:
        #print(e)
        pass

    
    try:
        job_link="https://indeed.com"+job.parent['href']
    except:
        pass
    
    try:
        job_company_name=str(job.find("span",{"class":"companyName"}).text).strip()
     
    except:
        pass

    try:
        company_link="https://indeed.com"+str(job.find("span",{"class":"companyName"}).find("a")['href']).strip()
    except:
        pass

    try:
        company_rating=str(job.find("span",{"class":"ratingNumber"}).text).strip()
    except:
        pass

    try:
        job_location=str(job.find("div",{"class":"companyLocation"}).text).strip()
    except:
        pass

    try:
        shortDescription=str(job.find("div",{"class":"job-snippet"}).text).strip()
    except:
        pass

    try:
        posting_date=str(job.find("span",{"class":"date"}).text).strip()
    except:
        pass

    try:
        salary_text=str(job.find("svg",{"aria-label":"Salary"}).parent.text)
    except:
        pass

    job_dict={"Company_Symbol":company_id,"SearchCompany":company,"JobType":job_type,"Country":search_country,
    'SearchLink':indeed_template,"JobTitle":job_title,"JobLink":job_link,"JobCompany":job_company_name,
    "CompanyLink":company_link,"CompanyRating":company_rating,"JobLocation":job_location,
    "ShortDescription":shortDescription,"PostingDate":posting_date,"Salary":salary_text}
    return job_dict



def fetch_page_jobs(driver):
    soup=BeautifulSoup(driver.page_source,'html5lib')
    jobs=soup.find_all("div",{"class":"slider_container"})
    jobs_list=list()
    #print(len(jobs))
    for job in jobs:
        #print(job)
        job_dict=fetch_job_info(job)
        jobs_list.append(job_dict)
    return jobs_list





In [109]:
company_stop_words=['corporation','inc.','limited','.com','company','incorporated','ltd.','co.']
special_symbol_company_mapping={
    'goog':'google',
    'googl':'google',
    'nsrgy':'nestle'
}

In [119]:
def clean_company_name(company_stop_words,company):
    company=company.replace(","," ").lower()
    for stop_word in company_stop_words:
        company=company.replace(stop_word,"")
    company=" ".join(company.split(" ")[:4]) # Fetching first 4 words from company name
    company=company.strip()

    return company


def save_list_to_csv(jobs_list,csv_file_name):
    table_cols=['Company_Symbol', 'SearchCompany', 'JobType', 'Country', 'SearchLink',
       'JobTitle', 'JobLink', 'JobCompany', 'CompanyLink', 'CompanyRating',
       'JobLocation', 'ShortDescription', 'PostingDate', 'Salary']
    df=pd.DataFrame(jobs_list)[table_cols]
    try:
        df_old=pd.read_csv(csv_file_name)[table_cols]
        print("File already exists. # jobs already present:",df_old.shape[0])
        print("Appending new job. Count = ",df.shape[0])
        df=pd.concat([df_old,df])
    except:
        pass
    df['JobType']=df['JobType'].str.replace(" ","NRML")
    df=df.drop_duplicates(subset=['JobLink'])
    df.to_csv(csv_file_name)
    print("Checkpoint saved. Total jobs:",df.shape[0])

def detect_captcha():
    try:
        driver.find_element_by_xpath("//div[@class='h-captcha']")
        print("Captcha Detected!")
        return 1
    except Exception as e:
        return 0

In [121]:
fast_forward=1
csv_file_name="Jobsmk3.csv"
start_count=247
df_already_fetched=pd.read_csv(csv_file_name)
symbols_fetched=list(df_already_fetched['Company_Symbol'].drop_duplicates())

#Devise a method to get companies only and not match by description
master_jobs_list=list()
job_types=['Internship',' ']
search_country=quote("United States")
num_pages=3
for i in range(start_count,df_companies_filtered.shape[0]):

    if(detect_captcha()):
        break

    clear_output()
    print(f"[{i+1}/{df_companies_filtered.shape[0]}]")
    company_id=str(df_companies_filtered['symbol'][i]).strip()
    if(company_id in symbols_fetched and fast_forward==1):
        print("Symbol Already Fetched. Skipping.")
        continue
    if((company_id.lower().strip()) in special_symbol_company_mapping):
        company=special_symbol_company_mapping[(company_id.lower().strip())]
    else:
        company=str(df_companies_filtered['companyName'][i])

    company=clean_company_name(company_stop_words, company)
    
    print("_________________________________________________")
    print("Searching jobs for:",company_id," | ",company,"\n")
    print("_________________________________________________")
    for job_type in job_types:
        print("\nJOB TYPE:",job_type)
        
        company_jobtype=company+" "+job_type
        company_jobtype=company_jobtype.strip()
        company_jobtype=quote(company_jobtype)
        for page in range(0,num_pages):
            page_count=page*10
            
            indeed_template=f"https://www.indeed.com/jobs?q={company_jobtype}&l={search_country}&start={page_count}"
            print("URL:",indeed_template)
            driver.get(indeed_template)
            time.sleep(random.uniform(1,4))
            soup=BeautifulSoup(driver.page_source,'html5lib')
            jobs=fetch_page_jobs(driver)
            master_jobs_list+=jobs
            print("Jobs fetched on page:",len(jobs))
            print("Jobs Fetched Till Now:",len(master_jobs_list))
            if(len(jobs)<=9):
                print("Bedrock hit, number of jobs fetched:",len(jobs))
                break

            
            
    time.sleep(random.uniform(3,10))
    if(i%50==0 and i!=0):
        try:
            save_list_to_csv(master_jobs_list,csv_file_name)
        except Exception as e:
            print("Exception while saving.",e)
        time.sleep(random.uniform(30,60))
try:
    save_list_to_csv(master_jobs_list,csv_file_name)
except Exception as e:
    print(e)

Captcha Detected!
"None of [Index(['Company_Symbol', 'SearchCompany', 'JobType', 'Country', 'SearchLink',\n       'JobTitle', 'JobLink', 'JobCompany', 'CompanyLink', 'CompanyRating',\n       'JobLocation', 'ShortDescription', 'PostingDate', 'Salary'],\n      dtype='object')] are in the [columns]"


In [ ]:
save_list_to_csv(master_jobs_list,csv_file_name)

KeyError: "None of [Index(['Company_Symbol', 'SearchCompany', 'JobType', 'Country', 'SearchLink',\n       'JobTitle', 'JobLink', 'JobCompany', 'CompanyLink', 'CompanyRating',\n       'JobLocation', 'ShortDescription', 'PostingDate', 'Salary'],\n      dtype='object')] are in the [columns]"

In [ ]:
#Drop duplicates on job link and check
df=pd.DataFrame(master_jobs_list).drop_duplicates(subset='JobLink')
df['JobType']=df['JobType'].str.replace("","NRML")
df.to_csv("Jobsmk3.csv")

(6156, 14)

<h2>Fields to fetch for every job</h2>
<hr><hr>
<h3>Input</h3>
<hr><hr>
<li> Company ID
<li> Company Name
<li> Job Type
<li> search_country
<li> Search Link
<hr><hr>
<h3>Searched</h3>
<hr><hr>
<li> jobLink
<li> Title
<li> indeedCompanyName
<li> rating
<li> job location
<li> short description
<li> payPerHour
<li> PostingDate


'AT%20%26%20T'